In [1]:
# this script removes "test" tweets from "dev" and "train" to reduce overfitting
# prouces cleaned 'dev' and 'train' files to accompany 'test'

In [91]:
train_con_fi = '../../data/Semeval_2020_task9_data/Spanglish/train.conll'
sp_test_fi = '../../data/Semeval_2020_task9_data/Spanglish/Spanglish_test_conll_labeled.txt'
sp_dev_fi = '../../data/Semeval_2020_task9_data/Spanglish/Spanglish_dev.conll'
sp_train_con_fi = '../../data/Semeval_2020_task9_data/Spanglish/Spanglish_train.conll'

In [92]:
def structure_data(fi):
    print('Now on {}'.format(fi.split('//')[-1]))
    with open(fi, 'r', encoding='utf-8') as f:
        lines = [x.replace('\n', '') for x in f.readlines()]
        
    d = {}
    cur_tweet = []
    cur_num = '1'  # all files start with 1
    cur_sentiment = 'positive'   # all files start positive
    for i in range(1, len(lines)):
        l = lines[i]
        data = l.split('\t')
        if l == '' or l == '\t':
            
            tweet_tokes = [x[0].lower() for x in cur_tweet]
            tweet_langs = [x[1] for x in cur_tweet]
            
            # unlabeled doesn't have sentiments
            if 'unlabeled' in fi:
                d[cur_num] = {'tweet_tokens':tweet_tokes, 'tweet_langs':tweet_langs}
            else:
                d[cur_num] = {'tweet_tokens':tweet_tokes, 'tweet_langs':tweet_langs,'sentiment':cur_sentiment}
        elif data[0].startswith('# sent_enum'):
            cur_num = data[0].split('=')[-1].strip()
            cur_sentiment = data[1]
            cur_tweet = []
        
        # other format
        elif data[0] == 'meta':
            if 'labeled' in fi:
                cur_num = data[1]
                cur_tweet = []
            else:
                cur_num = data[0].split('=')[-1].strip()
                cur_sentiment = data[1]
                cur_tweet = []
        
        else:
            cur_tweet.append(data)
    return d


In [93]:
train_con_d = structure_data(train_con_fi)
sp_test_d = structure_data(sp_test_fi)
sp_dev_d = structure_data(sp_dev_fi)
sp_train_con_d = structure_data(sp_train_con_fi)

Now on ../../data/Semeval_2020_task9_data/Spanglish/train.conll
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_test_conll_labeled.txt
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_dev.conll
Now on ../../data/Semeval_2020_task9_data/Spanglish/Spanglish_train.conll


In [94]:
train_outs = []
# train file
to_remove = []
for k in train_con_d:
    tweet_tokens = train_con_d[k]['tweet_tokens']
    tweet_langs = train_con_d[k]['tweet_langs']
    sentiment = train_con_d[k]['sentiment']
    
    for j in sp_test_d:
        test_tweet_tokens = sp_test_d[j]['tweet_tokens']
        if tweet_tokens == test_tweet_tokens:
            to_remove.append(k)
            break

for k in train_con_d:
    if k not in to_remove:
        val = train_con_d[k]['sentiment']

        train_outs.append('{}\t{}\t{}'.format('meta',k,val))
        tts = train_con_d[k]['tweet_tokens']
        tls = train_con_d[k]['tweet_langs']
        for i in range(len(tts)):
            train_outs.append('{}\t{}'.format(tts[i],tls[i]))
        train_outs.append('')
            

In [95]:
len(to_remove)

2153

In [96]:
dev_outs = []
# dev file
to_remove_dev = []
for k in sp_dev_d:
    tweet_tokens = sp_dev_d[k]['tweet_tokens']
    tweet_langs = sp_dev_d[k]['tweet_langs']
    sentiment = sp_dev_d[k]['sentiment']
    
    for j in sp_test_d:
        test_tweet_tokens = sp_test_d[j]['tweet_tokens']
        if tweet_tokens == test_tweet_tokens:
            to_remove_dev.append(k)
            break

for k in sp_dev_d:
    if k not in to_remove_dev:
        val = sp_dev_d[k]['sentiment']

        dev_outs.append('{}\t{}\t{}'.format('meta',k,val))
        tts = sp_dev_d[k]['tweet_tokens']
        tls = sp_dev_d[k]['tweet_langs']
        for i in range(len(tts)):
            dev_outs.append('{}\t{}'.format(tts[i],tls[i]))
        dev_outs.append('')
print(len(to_remove_dev))

0


In [97]:
sp_train_outs = []
# sp_train file
sp_to_remove = []
for k in sp_train_con_d:
    tweet_tokens = sp_train_con_d[k]['tweet_tokens']
    tweet_langs = sp_train_con_d[k]['tweet_langs']
    sentiment = sp_train_con_d[k]['sentiment']
    
    for j in sp_test_d:
        test_tweet_tokens = sp_test_d[j]['tweet_tokens']
        if tweet_tokens == test_tweet_tokens:
            sp_to_remove.append(k)
            break

for k in sp_train_con_d:
    if k not in sp_to_remove:
        val = sp_train_con_d[k]['sentiment']

        sp_train_outs.append('{}\t{}\t{}'.format('meta',k,val))
        tts = sp_train_con_d[k]['tweet_tokens']
        tls = sp_train_con_d[k]['tweet_langs']
        for i in range(len(tts)):
            sp_train_outs.append('{}\t{}'.format(tts[i],tls[i]))
        sp_train_outs.append('')
            

In [98]:
len(sp_to_remove)

0

In [99]:
train_con_fi_clean = '../../data/Semeval_2020_task9_data/Spanglish/train_clean.conll'
with open(train_con_fi_clean, 'w', encoding='utf-8') as f:
    for x in train_outs:
        f.write('{}\n'.format(x))